In [29]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
# for keras
from classification_models.keras import Classifiers

# for tensorflow keras
from classification_models.tfkeras import Classifiers

In [30]:
fake_train_path = "./Dataset/Train/small/smaller/Fake_small_15000"
real_train_path = "./Dataset/Train/small/smaller/Real_small_15000"
train_path = './Dataset/Train/small/smaller'

In [31]:
len(os.listdir(fake_train_path))

15001

In [32]:
train_dataset = image_dataset_from_directory(
    train_path,
    image_size=(100,100),
    batch_size=32,
    label_mode='binary',
    shuffle=True,
    seed=45
)

Found 30002 files belonging to 2 classes.


In [33]:
# for images, labels in train_dataset.take(1):
#     break
# plt.figure(figsize=(15, 15))
# for i in range(10):
#     plt.subplot(8, 8, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(labels[i].numpy())

In [34]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [35]:
inceptionresnetv2, preprocess_input = Classifiers.get('inceptionresnetv2')
base_model = inceptionresnetv2(include_top = False, input_shape=(100, 100, 3), weights='imagenet')

In [36]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 1, 1, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_2   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 1024)              1573888   
                                                                 
 dense_7 (Dense)             (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 1)                 513       
                                                                 
Total params: 56,435,937
Trainable params: 56,375,393


In [39]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [40]:
train_dataset = train_dataset.map(process)

In [41]:
for images, labels in train_dataset.take(1):
    break

max(images[0][0][0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.47392786>

In [42]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
938/938 [==============================] - 262s 256ms/step - loss: 0.1649 - accuracy: 0.9357
Epoch 2/5
938/938 [==============================] - 210s 224ms/step - loss: 0.0814 - accuracy: 0.9706
Epoch 3/5
938/938 [==============================] - 196s 208ms/step - loss: 0.0626 - accuracy: 0.9760
Epoch 4/5
938/938 [==============================] - 195s 208ms/step - loss: 0.0565 - accuracy: 0.9791
Epoch 5/5
938/938 [==============================] - 195s 208ms/step - loss: 0.0514 - accuracy: 0.9811


In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = './Just as it is/fake_146.jpg'  # Replace with your image path
img = image.load_img(img_path, target_size=(100, 100))  # Resize to match model input shape
img_array = image.img_to_array(img)
img_array = img_array/255.0
# img_array = tf.image.per_image_standardization(img_array)  # Standardize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict with the model
prediction = model.predict(img_array)

# Interpretation of the result
if prediction[0] > 0.5:
    print(f"The image is predicted to be a real with a confidence of {prediction[0][0]*100:.2f}%")
else:
    print(f"The image is predicted to be deepfake with a confidence of {(1-prediction[0][0])*100:.2f}%")


1/1 [==============================] - 4s 4s/step
The image is predicted to be a real with a confidence of 86.09%


In [46]:
test_path = './1000_videos/test'
test_dataset = image_dataset_from_directory(
    test_path,
    image_size=(100,100),
    batch_size=32,
    label_mode='binary',
    shuffle=True
)

loss, accuracy = model.evaluate(train_dataset)

Found 2400 files belonging to 2 classes.
938/938 [==============================] - 66s 67ms/step - loss: 0.0620 - accuracy: 0.9709


In [ ]:
# loss: 0.0620 - accuracy: 0.9709